# 📊 QEPC NBA Backtest Notebook

This notebook measures how well the QEPC NBA engine is performing over time.

It focuses on:
- 🗓️ Defining a backtest date range
- 🧮 Running `run_season_backtest(...)` to simulate all games in that range
- 📈 Summarizing accuracy & spread error
- 🧨 Inspecting the worst misses
- 🧹 Filtering to NBA–vs–NBA only (no exhibitions)
- 🌌 (Optional) Script / total-error analysis


------

## 🧩 1. Environment & Setup


In [1]:
# Universal QEPC header for this notebook

import os
import sys
from pathlib import Path

try:
    from notebook_context import *  # try direct import first
except ModuleNotFoundError:
    cwd = Path.cwd()
    candidate_roots = [cwd, cwd.parent, cwd.parent.parent]

    found_root = None
    for root in candidate_roots:
        if (root / "notebook_context.py").exists():
            found_root = root
            break

    if found_root is None:
        raise ModuleNotFoundError(
            f"Could not find notebook_context.py from {cwd}. "
            "Try opening this notebook from inside your qepc_project folder."
        )

    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    from notebook_context import *

# Fallback for project_root if notebook_context didn't define it
try:
    project_root
except NameError:
    project_root = Path.cwd()

print("Project root:", project_root)


[QEPC Paths] Project Root set: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project
[QEPC] Autoload complete.
[QEPC] Root Shim Restored. Forwarding to qepc.autoload...
Project root: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project


-----

## 📅 2. Select Backtest Range


In [2]:
# 2. Backtest Date Range & Filters (Custom start → today, no widgets)

import pandas as pd
from datetime import date

# 👇 Edit this line when you want a different start date
BACKTEST_START_DATE = date(2024, 11, 24)

# End date is always "today"
BACKTEST_END_DATE = date.today()

print("Backtest date range:",
      BACKTEST_START_DATE.isoformat(), "→", BACKTEST_END_DATE.isoformat())


Backtest date range: 2024-11-24 → 2025-11-27


-----

## 🧮 3. Initiate Backtest


In [3]:
# 3. Run Backtest for Selected Range (start → today)

import pandas as pd
import qepc_autoload as qa
from pathlib import Path
from qepc.backtest.backtest_engine import run_season_backtest

# 3.1 Make sure we have a schedule DataFrame
try:
    schedule
    print("✅ Using existing 'schedule' from memory.")
except NameError:
    print("ℹ️ 'schedule' not found; loading from qa.load_nba_schedule() ...")
    schedule = qa.load_nba_schedule()
    print("   Loaded schedule with", len(schedule), "games.")

# 3.2 Make sure we have team_strengths_for_lambda
try:
    team_strengths_for_lambda
    print("✅ Using existing 'team_strengths_for_lambda'.")
except NameError:
    print("ℹ️ 'team_strengths_for_lambda' not found; building from strengths_v2...")

    from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
    try:
        from qepc.sports.nba.strengths_v2 import apply_injury_overrides
    except ImportError:
        apply_injury_overrides = None

    # Base strengths (no injuries yet)
    base_strengths = calculate_advanced_strengths()
    print("   Base strengths rows:", len(base_strengths))

    # Try to apply your merged injury overrides if they exist
    injuries_master_path = project_root / "data" / "Injury_Overrides_MASTER.csv"

    if apply_injury_overrides and injuries_master_path.exists():
        try:
            injuries_master = pd.read_csv(injuries_master_path)
            print("   Loaded injuries master with", len(injuries_master), "rows.")
            team_strengths_for_lambda = apply_injury_overrides(base_strengths, injuries_master)
            print("   Applied injury overrides.")
        except Exception as e:
            print("   ⚠️ Could not apply injury overrides, using base strengths instead.")
            print("   Reason:", e)
            team_strengths_for_lambda = base_strengths
    else:
        print("   No usable injury master found; using base strengths only.")
        team_strengths_for_lambda = base_strengths

# 3.3 Make sure we have GLOBAL_LAMBDA_SCALE
try:
    GLOBAL_LAMBDA_SCALE
    print("✅ Using existing GLOBAL_LAMBDA_SCALE:", GLOBAL_LAMBDA_SCALE)
except NameError:
    GLOBAL_LAMBDA_SCALE = 1.0
    print("ℹ️ GLOBAL_LAMBDA_SCALE not found; defaulting to 1.0")

# 3.4 Convert date range and run the backtest
start_date = pd.to_datetime(BACKTEST_START_DATE)
end_date = pd.to_datetime(BACKTEST_END_DATE)

print(f"\nRunning QEPC backtest from {start_date.date()} to {end_date.date()}")

backtest_long = run_season_backtest(
    schedule,
    team_strengths_for_lambda,
    start_date,
    end_date,
    GLOBAL_LAMBDA_SCALE,
)

print("Games simulated in this range:", len(backtest_long))
display(backtest_long.head())


ℹ️ 'schedule' not found; loading from qa.load_nba_schedule() ...
[QEPC NBA Sim] Successfully loaded and parsed 771 games from original format.
   Loaded schedule with 771 games.
ℹ️ 'team_strengths_for_lambda' not found; building from strengths_v2...
Built advanced team strengths from Team_Stats.csv
  Teams: 48
  ORtg range: 85.7 – 111.9
  DRtg range: 95.2 – 131.7
  Pace range: 85.7 – 111.9
  Volatility range: 8.00 – 14.00
   Base strengths rows: 48
   No usable injury master found; using base strengths only.
ℹ️ GLOBAL_LAMBDA_SCALE not found; defaulting to 1.0

Running QEPC backtest from 2024-11-24 to 2025-11-27


TypeError: run_season_backtest() takes 2 positional arguments but 5 were given

----

## 🏁 4. Global Summary (All Games in Range)


In [ ]:
# 4. Global Summary (All Games in Selected Range)

if "backtest_long" not in globals():
    raise RuntimeError("Run Cell 3 (Run Backtest) before the summary.")

total_games = len(backtest_long)
accuracy_pct = backtest_long["Correct_Pick"].mean() * 100
spread_mae = backtest_long["Spread_Error"].abs().mean()

print("========================================")
print("🏆 QEPC BACKTEST SUMMARY")
print("========================================")
print(f"📅 Date Range: {start_date.date()} → {end_date.date()}")
print(f"🏀 Games Simulated: {total_games}")
print(f"✅ Overall Accuracy: {accuracy_pct:.2f}%")
print(f"🎯 Avg Spread Error (MAE): {spread_mae:.2f} points")
print("========================================")


----

## 🧹 5. Clean NBA-Only View (Filter Out Exhibitions)


In [ ]:
# 5. Clean NBA-Only View (Filter Out Exhibitions)

NBA_TEAMS = [
    "Atlanta Hawks", "Boston Celtics", "Brooklyn Nets", "Charlotte Hornets",
    "Chicago Bulls", "Cleveland Cavaliers", "Dallas Mavericks", "Denver Nuggets",
    "Detroit Pistons", "Golden State Warriors", "Houston Rockets", "Indiana Pacers",
    "Los Angeles Clippers", "Los Angeles Lakers", "Memphis Grizzlies", "Miami Heat",
    "Milwaukee Bucks", "Minnesota Timberwolves", "New Orleans Pelicans", "New York Knicks",
    "Oklahoma City Thunder", "Orlando Magic", "Philadelphia 76ers", "Phoenix Suns",
    "Portland Trail Blazers", "Sacramento Kings", "San Antonio Spurs", "Toronto Raptors",
    "Utah Jazz", "Washington Wizards",
]

backtest_clean = backtest_long[
    backtest_long["Away Team"].isin(NBA_TEAMS)
    & backtest_long["Home Team"].isin(NBA_TEAMS)
].copy()

print(f"Original games: {len(backtest_long)}, After NBA-only filter: {len(backtest_clean)}")

acc_clean = backtest_clean["Correct_Pick"].mean() * 100
mae_clean = backtest_clean["Spread_Error"].abs().mean()

print("========================================")
print("🏆 CLEAN NBA-ONLY BACKTEST")
print("========================================")
print(f"🏀 Games Simulated: {len(backtest_clean)}")
print(f"✅ Overall Accuracy: {acc_clean:.2f}%")
print(f"🎯 Avg Spread Error: {mae_clean:.2f} points")


-----

## 🧨 6. Biggest Misses (Spread Error Analysis)


In [ ]:
# 6. Biggest Misses (Spread Error Analysis)

worst = backtest_clean.reindex(
    backtest_clean["Spread_Error"].abs().sort_values(ascending=False).index
)

cols = [
    "Date",
    "Away Team", "Home Team",
    "Expected_Spread", "Actual_Spread", "Spread_Error",
    "Away_Win_Prob", "Home_Win_Prob",
    "Sim_Away_Score", "Actual_Away_Score",
    "Sim_Home_Score", "Actual_Home_Score",
    "Correct_Pick",
]
cols = [c for c in cols if c in worst.columns]

print("Top 15 biggest spread misses (NBA vs NBA only):")
display(worst[cols].head(15))


----

## 📦 7. Spread Error Buckets (How Often & How Bad)


In [ ]:
# 7. Spread Error Buckets (How Often & How Bad)

abs_err = backtest_clean["Spread_Error"].abs()

bins = [0, 5, 10, 15, 20, np.inf]
labels = ["0–5", "5–10", "10–15", "15–20", "20+"]

backtest_clean["Spread_Error_Bucket"] = pd.cut(
    abs_err, bins=bins, labels=labels, right=False
)

bucket_counts = backtest_clean["Spread_Error_Bucket"].value_counts().sort_index()
bucket_pct = (bucket_counts / len(backtest_clean) * 100).round(2)

spread_bucket_summary = pd.DataFrame({
    "Games": bucket_counts,
    "Percent": bucket_pct
})

print("Spread error distribution (absolute error in points):")
display(spread_bucket_summary)


----

## 🌌 8. Optional: Total Error & Script Classification (GRIND / BASE / CHAOS)


-----

In [ ]:
# 8. Optional: Total Error & Script Classification (GRIND / BASE / CHAOS)

df = backtest_clean.copy()

# 8.1 Compute simulated and actual totals
df["Sim_Total"] = df["Sim_Home_Score"] + df["Sim_Away_Score"]
df["Actual_Total"] = df["Actual_Home_Score"] + df["Actual_Away_Score"]

# Drop games where we effectively didn't simulate (Sim_Total == 0)
before = len(df)
df = df[df["Sim_Total"] > 0]
after = len(df)
print(f"Dropped {before - after} games with Sim_Total == 0.")
print(f"Remaining games for script analysis: {after}")

# 8.2 Total error
df["Total_Error"] = df["Actual_Total"] - df["Sim_Total"]

# Thresholds for GRIND / BASE / CHAOS (can be tuned)
grind_thresh = -15   # 15+ pts under model total → GRIND
chaos_thresh = 15    # 15+ pts over model total → CHAOS

def classify_script(row):
    if row["Total_Error"] <= grind_thresh:
        return "GRIND"
    elif row["Total_Error"] >= chaos_thresh:
        return "CHAOS"
    else:
        return "BASE"

df["Script_ExPost"] = df.apply(classify_script, axis=1)

print("\nSample of script labels:")
display(
    df[["Date", "Away Team", "Home Team", "Sim_Total", "Actual_Total", "Total_Error", "Script_ExPost"]]
    .head()
)

# 8.3 Script-level summary
script_summary = df.groupby("Script_ExPost").agg(
    Games=("Total_Error", "count"),
    Avg_Total_Error=("Total_Error", "mean"),
    Avg_Abs_Total_Error=("Total_Error", lambda x: x.abs().mean()),
    Avg_Spread_Error=("Spread_Error", "mean"),
    Avg_Abs_Spread_Error=("Spread_Error", lambda x: x.abs().mean()),
)

script_summary["Percent"] = (script_summary["Games"] / len(df) * 100).round(2)

print("\nEx-post script distribution (based on totals):")
display(script_summary)


## Global Lambda Export

In [ ]:
# === QEPC Global Lambda Calibration Export ===
# Set this based on your backtest experiments.
# For now you can leave it at 1.0; later you can change it to (for example) 0.97.

scale_factor = 1.0  # 👈 tweak this number when you want to shrink/boost totals

import json
from pathlib import Path

calibration = {
    "global_lambda_scale": float(scale_factor),
}

calib_path = project_root / "data" / "qepc_calibration.json"
calib_path.parent.mkdir(parents=True, exist_ok=True)

with open(calib_path, "w") as f:
    json.dump(calibration, f, indent=2)

print("✅ Saved calibration file to:", calib_path)
print("   Contents:", calibration)
